<a href="https://colab.research.google.com/github/MarkovMarkowitz/MarkovMarkowitz/blob/main/MarkovChainGainProbabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Two Absorbing States: Gambler's Ruin**

Now, consider the following situation. A gambler bets on the outcome of a sequence of independent fair coin tosses. With each heads, the gambler gains one dollar. With each tails, the gambler loses one dollar. The gambler stops betting after reaching a fortune of $\overline{S}$ dollars or after emptying their pockets.

*   What are the probabilities of each stopping outcome?
*   How long will it take for the gambler, in expectation, to arrive at one of the stopping outcomes?

To answer these questions, we can model this setting as a Markov chain on the state space $\mathcal{S}\in\{0,1,...,\overline{s}\}$. The gambler starts with initial money $k\in\mathcal{S}$, and $s_t$ represents the money in the gambler's pocket at time $t$. Thus, we have that, for $0\lt s_t \lt \overline{s}$:

*   $\mathbb{P}(s_{t+1}=s_t+1|s_{t})=0.5$
*   $\mathbb{P}(s_{t+1}=s_t-1|s_{t})=0.5$

States 0 and $\overline{s}$ are absorbing states because any sequence of draws from the Markov chain stops after reaching any of those situations. Alternatively, we can think that $\mathbb{P}(s_{t+1}=s_t|s_{t}=\overline{s})=\mathbb{P}(s_{t+1}=s_t|s_{t}=0)=1$. We can then represent the $(\overline{s}+1)\times(\overline{s}+1)$ transition matrix as:
$$
\begin{align}
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 &\cdots & 0 \\
0.5 & 0 & 0.5 & 0 & 0 &\cdots & 0 \\
0 & 0.5 & 0 & 0.5 & 0 & \cdots & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots& \cdots & \vdots \\
0 & 0 & 0 & 0.5 & 0 & 0.5 & 0 \\
0 & 0 & 0 & 0 & 0.5 & 0 & 0.5 \\
0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
\end{align}
$$
Before solving this with math, let's see some Monte Carlo simulation results (in this example, the gambler stops betting after reaching a fortune of 5 USD and starts with 1 USD):

In [37]:

# LIBRARIES WE USE IN THE NOTEBOOK
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import rand, seed
from scipy.stats import norm
# seed random number generator
seed(123)

p= 0.5

TARGET_PURSE = 11
INIT_PURSE = 10

N_STATES = TARGET_PURSE + 1

S = np.zeros((N_STATES, 1))
P = np.zeros((N_STATES, N_STATES))

P[0, 0] = 1.0
P[N_STATES - 1, N_STATES - 1] = 1.0

for ii in range(1, N_STATES - 1):
    for jj in range(0, N_STATES):
        if jj == ii - 1 :
          P[ii, jj] = 1-p
        if jj == ii + 1 :
          P[ii, jj] = p

print("Transition matrix:\n", P)

Transition matrix:
 [[1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


In [38]:
N_HISTORIES = 10000  # number of histories or simulations
LEN_HIST = 100  # Length of each simulation
histories = np.zeros((N_HISTORIES, LEN_HIST))
histories[:, 0] = INIT_PURSE * np.ones(N_HISTORIES)
randarray = rand(N_HISTORIES, LEN_HIST)

for i in range(0, N_HISTORIES):
    for j in range(1, LEN_HIST):
        histories[i, j] = (
            histories[i, j - 1] + (randarray[i, j] >= 0.5) - (randarray[i, j] < 0.5)
        )
        if histories[i, j] == TARGET_PURSE or histories[i, j] < 1:
            histories[i, j + 1 : LEN_HIST + 1] = histories[i, j]  # noQA E203
            break

target_num = np.sum(np.max(histories, axis=1) == TARGET_PURSE)

end_gamble = np.zeros(N_HISTORIES)
end_gamble_sum = 0

for i in range(0, N_HISTORIES):
    if np.max(histories[i, :]) == TARGET_PURSE:
        where_gamble_ends_T = np.where((histories[i, :] == TARGET_PURSE))
        end_gamble[i] = where_gamble_ends_T[0][0]
        end_gamble_sum += 1
    elif np.min(histories[i, :]) < 1:
        where_gamble_ends_0 = np.where((histories[i, :] < 1))
        end_gamble[i] = where_gamble_ends_0[0][0]
        end_gamble_sum += 1
    else:
        end_gamble[i] = 0.0

broke_num = np.sum(np.min(histories, axis=1) < 1)

print(
    "Probability of getting the target:",
    target_num / N_HISTORIES,
    "\nProbability of losing all the money:",
    broke_num / N_HISTORIES,
)
print(
    "Expected time until reaching a stopping result:",
    np.sum(end_gamble) / end_gamble_sum,
    "\nTotal number of simulations:",
    end_gamble_sum,
)

Probability of getting the target: 0.9065 
Probability of losing all the money: 0.0875
Expected time until reaching a stopping result: 9.10010060362173 
Total number of simulations: 9940
